In [ ]:
import quantz_repo
import pandas as pd
from pandas import DataFrame, Series
import tushare  as ts
import akshare as ak
from quantz_repo import utils,QuantzException
from quantz_repo.utils import df_2_mongo, mongo_2_df, yyyymmdd_2_int
from quantz_repo.model import BasicStockInfoItem, BasicTradingInfoItem, UsJoblessInitialClaimItem
from quantz_repo.models import MarketWidthItem
from quantz_repo.utils import get_next_day_in_YYYYMMDD, timestamp_2_YYYYMMDD, round_half_up,now_2_YYYYMMDD
from quantz_repo.utils import Fred
from quantz_repo import initialize_daily_trading_info, update_daily_trading_info_for,get_stock_basics, get_trade_dates_between
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
from io import StringIO
import datetime

import mongoengine
mongoengine.connect('quantz')
# mongoengine.connect('quant_test', host='192.168.70.200', port=27017)

In [ ]:
'''
如何初始化过去的市场宽度？ 
1）从何时开始? 20
2）是否所有时间内行业分类的成员是相同的？若不相同如何处理？ 无法拿到当时的数据，按照当前分类处理
3）某行业分类内的股票在某段时间之前可能还没有上市，应该如何处理？ 按照当时实际股票个数来计算分数，可能导致很久之前的某个时间，某个股票不会被计算到任意的分数中，接受现状，TODO:以后有数据再更新
4) 按照调入日期计算，对于一段时间之前的值计算肯定是不准确的，只能记录当前值逐步积累
'''
def rank_industry_at(index_code:str, industry_name:str, level:str, trade_date:int, force:bool=False):
    '''
    计算某个行业在某天的分值，
    index_code: 行业代码，申万的
    trade_date: 交易日 
    force: 是否强制更新
    '''
    print('rank %s on %s' % (index_code, timestamp_2_YYYYMMDD(trade_date)))
    the_item = MarketWidthItem.objects(index_code__iexact=index_code,trade_date=trade_date)
    if force:
        # 删除已有数据
        the_item.delete()
        print('Rank deleted')
    elif the_item.count() > 0:
        # print('Already ranked for %s on %s，Skipping' % (index_code, timestamp_2_YYYYMMDD(trade_date)));
        return
    members = quantz_repo.get_industrial_classfication_members(index_code,trade_date=trade_date)
    print('members')
    print(members.shape)
    if not members.empty:
        m_list = list(members['con_code'].array)
        # print(m_list)
        member_stocks_df = mongo_2_df(BasicTradingInfoItem.objects(ts_code__in=m_list, trade_date=trade_date))
        if member_stocks_df.empty:
            #raise QuantzException('Faile to get trading info for %s on %s(%d), try update trading info first' % (index_code, timestamp_2_YYYYMMDD(trade_date), trade_date))
            print('Faile to get trading info for %s on %s(%d), try update trading info first' % (index_code, timestamp_2_YYYYMMDD(trade_date), trade_date))
            return
        # print('members stock trading info')
        # print(member_stocks_df.shape)
        rank = 0
        try:
            gt_df = member_stocks_df[member_stocks_df['close']>=member_stocks_df['ma_close_20']]
        # print('strong items')
        # print(gt_df.shape)
            rank = round_half_up(100 * gt_df.shape[0] / member_stocks_df.shape[0])
        except Exception  as e:
            rank = 0
            print('Failed to rank %s on %s(%d) cause:%s, you should check the data' %  (index_code, timestamp_2_YYYYMMDD(trade_date), trade_date, e))
        # print('rank: %f' % rank)
        MarketWidthItem(index_code=index_code, industry_name=industry_name,industry_level=level,rank=rank,trade_date=trade_date).save()
    else:
        raise QuantzException('None memeber in %s before %s' % (index_code, timestamp_2_YYYYMMDD(trade_date)))
    
    

# rank_industry_at(index_code='801020.SI', industry_name='随便mingzi', trade_date=1606060800000, level='L2', force=True)

In [ ]:
rank_industry_at(index_code='801770.SI', industry_name='zzz', trade_date=1586793600000, level='L1', force=True)

In [ ]:
def rank_industry_level_at(level:str, trade_date:int, force:bool = False):
    industries_df = quantz_repo.get_industrial_classifications(level=level)
    if industries_df is  None or industries_df.empty:
        print('Failed to get industry classification for %s' % level) 
    else:
        for i in industries_df.itertuples():
            rank_industry_at(i.index_code, i.industry_name, level, trade_date, force)

In [ ]:
# 获取一级行业分类列表
industries_df = quantz_repo.get_industrial_classifications(level='L1')
industries_df.head(8)

In [ ]:
# 获取行业内股票列表
stock_array = list(quantz_repo.get_industrial_classfication_members('801770.SI')['con_code'].array)
stock_array

In [ ]:
BasicTradingInfoItem.objects(ts_code__in = stock_array,trade_date=1605801600000).to_json()

In [ ]:
BasicTradingInfoItem.objects(__raw__ = {'ts_code':{'$in':stock_array},'trade_date':1605801600000}).to_json()

In [ ]:
rank_industry_level_at(level='L1',trade_date=1619625600000)

In [ ]:
rank_industry_level_at(level='L2',trade_date=1606060800000)

In [ ]:
rank_industry_level_at(level='L3',trade_date=1606060800000)

In [ ]:
def rank_industry_level_between(level='L1', since:str='20210101', end:str=now_2_YYYYMMDD()):
    trade_cal = quantz_repo.get_trade_dates_between(since,end)
    for trade_date in trade_cal.itertuples():
        rank_industry_level_at(level=level, trade_date=trade_date.cal_date)


In [ ]:
rank_industry_level_between()

In [ ]:
def rank_all_industry_between(since:str='20210427', end:str='20210506'):
    for level in ['L1', 'L2', 'L3']:
        rank_industry_level_between(level=level, since=since, end=end)

In [ ]:
rank_all_industry_between(since='20210427',end='20210506')

In [ ]:
quantz_repo.init_trade_calendar()

In [ ]:
quantz_repo.update_daily_trading_info()

In [ ]:
# quantz_repo.update_daily_trading_info()
quantz_repo.initialize_daily_trading_info()

In [ ]:
def update_industry_level(level:str,start:int, end:int):
    '''
    '''
    # 检查开始、截止时间是否在当前交易日历内,否则要求更新交易日

In [ ]:
# update_daily_trading_info_for('000003.SZ')

In [ ]:
# 获取股票基本信息
# quantz_repo.get_stock_basics()

In [ ]:
# 更定股票基本信息
quantz_repo.update_stock_basics()

In [ ]:
# 初始化行业分类
quantz_repo.initialize_industrial_classification()

In [ ]:
# 更新行业分类
quantz_repo.update_industry_classification()

In [ ]:
quantz_repo.get_us_initial_jobless()

In [ ]:
quantz_repo.get_us_wei()

In [ ]:
quantz_repo.get_us_ccsa()